# Resources
- https://stackoverflow.com/questions/46826773/how-can-i-merge-rows-by-same-value-in-a-column-in-pandas-with-aggregation-func
- https://realpython.com/pandas-merge-join-and-concat/
- https://stackoverflow.com/questions/53645882/pandas-merging-101
- https://stackoverflow.com/questions/22341271/get-list-from-pandas-dataframe-column-or-row
- 

In [122]:
import pandas as pd
import numpy as np
import json

In [140]:
files = [
    "A) Senarai Klinik Perubatan Swasta Berdaftar Yang Boleh Menjalankan Saringan COVID-19 Dalam Kawasan Premis",
    "B) Senarai Hospital Swasta Berlesen Yang Boleh Menjalankan Saringan COVID-19 Di Dalam Kawasan Premis",
    "Klinik 1Malaysia",
    "Klinik Desa",
    "Klinik Kesihatan Kerajaan",
    "Klinik Kesihatan Yang Ada Kerjasama Dengan Wakil Komuniti 2017",
    "Lokasi Klinik Kesihatan Swasta di Pulau Pinang",
    "Lokasi Pusat Kesihatan Awam di Negeri Pulau Pinang 2018",
    "Senarai Klinik Ibu Dan Anak Mengikut Negeri Dan Daerah",
    "hospital_data",
    "VAC",
]

In [141]:
def coord_norm(x):
    if type(x) is float:
        return str(x)
    return x

In [142]:
df_list = [pd.read_csv(f"CSV/{file}.csv").replace(np.nan, '', regex=True) for file in files]

In [143]:
# update ST hospital_data 
df_list[9]["ST"] = 1

In [144]:
# join dup
vac_hospital = pd.concat([df_list[9], df_list[10]])
vac_hospital["Latitude"] = vac_hospital["Latitude"].apply(coord_norm)
vac_hospital["Longitude"] = vac_hospital["Longitude"].apply(coord_norm)
vac_hospital = vac_hospital.groupby(vac_hospital['Name'], as_index=False).aggregate(
    {
        "Clinic":"min",
        "Hospital":"max",
        "ST":"max",
        "VAC":"max",
        "Latitude":"max",
        "Longitude":"max",
        "Address":"max",
        "Contact":"max",
        "State":"max",
    }
)

In [145]:
vac_hospital.tail()

,Name,Clinic,Hospital,ST,VAC,Latitude,Longitude,Address,Contact,State
684,University College TATI,0,0,0,1,,,Kemaman,,Terengganu
685,Wisma Belia,0,0,0,1,3.82478,103.28139,Kuantan,,Pahang
686,World Trade Centre KL (WTCKL),0,0,0,1,,,Kuala Lumpur,,W.P. Kuala Lumpur
687,Xcape Resort Tahan,0,0,0,1,,,Jerantut,,Pahang
688,Yayasan Toh Puan Zurina,0,0,0,1,,,Melaka Tengah,,Melaka


In [146]:
merge_df = pd.concat(df_list[:9], ignore_index=True)
merge_df = pd.concat([merge_df, vac_hospital], ignore_index=True)

In [147]:
print(merge_df["State"].unique())

['Johor' 'Kedah' 'Kelantan' 'Melaka' 'Negeri Sembilan' 'Pahang'
 'Pulau Pinang' 'Perak' 'Perlis' 'Selangor' 'Terengganu' 'Sabah' 'Sarawak'
 'Wp Kuala Lumpur & Putrajaya' 'Wp Labuan' 'Pulau pinang' 'Kuala lumpur'
 'WP Labuan' 'WP Kuala Lumpur' 'WP Putrajaya' 'Wp Kuala Lumpur'
 'Wp Putrajaya' 'W.P. Kuala Lumpur' 'W.P. Labuan' 'W.P. Putrajaya']


In [148]:
merge_df["State"].replace(to_replace=['Wp Kuala Lumpur & Putrajaya', 'WP Kuala Lumpur', 'Kuala lumpur', 'Wp Kuala Lumpur', 'W.P. Kuala Lumpur'], value="Kuala Lumpur", inplace=True)
merge_df["State"].replace(to_replace=['Pulau pinang'], value="Pulau Pinang", inplace=True)
merge_df["State"].replace(to_replace=['Wp Labuan', 'WP Labuan', 'W.P. Labuan'], value="Labuan", inplace=True)
merge_df["State"].replace(to_replace=['WP Putrajaya', 'Wp Putrajaya', 'W.P. Putrajaya'], value="Putrajaya", inplace=True)
merge_df["State"].replace(to_replace=['Jalan Changkat Jong'], value="Perak", inplace=True)

In [149]:
# uppercase all value in state
merge_df["State"] = merge_df["State"].str.upper()

In [150]:
merge_df.tail()

,index,Name,Clinic,Hospital,ST,VAC,Latitude,Longitude,Address,Contact,State
7080,NaN,University College TATI,0,0,0,1,,,Kemaman,,TERENGGANU
7081,NaN,Wisma Belia,0,0,0,1,3.82478,103.28139,Kuantan,,PAHANG
7082,NaN,World Trade Centre KL (WTCKL),0,0,0,1,,,Kuala Lumpur,,KUALA LUMPUR
7083,NaN,Xcape Resort Tahan,0,0,0,1,,,Jerantut,,PAHANG
7084,NaN,Yayasan Toh Puan Zurina,0,0,0,1,,,Melaka Tengah,,MELAKA


In [151]:
merge_df["Latitude"] = merge_df["Latitude"].apply(coord_norm)
merge_df["Longitude"] = merge_df["Longitude"].apply(coord_norm)

In [152]:
# drop index col
merge_df = merge_df.drop(columns=["index"])

In [153]:
dup_df = merge_df[merge_df.duplicated(['Name'], keep=False)]
dup_df

,Name,Clinic,Hospital,ST,VAC,Latitude,Longitude,Address,Contact,State
0,Klinik Lee Dan Surgeri,1,0,1,0,,,"45 (Ground Floor), Jalan Mutiara 1/2, Taman Mu...",,JOHOR
3,Klinik Yap & Partners,1,0,1,0,,,"(Ground Floor), 7523 & 7524, Jalan Enggang 19,...",,JOHOR
6,Poliklinik Penawar,1,0,1,0,,,"No. 104, Jalan Bintang, Taman Bintang 81400 Se...",,JOHOR
7,Poliklinik Penawar,1,0,1,0,,,"No. 16, Pusat Bandar, Bandar Penawar, 81900 Ko...",,JOHOR
9,Poliklinik Penawar,1,0,1,0,,,"No. 49, Jalan Medan Nusa Perintis 6, Taman Nus...",,JOHOR
...,...,...,...,...,...,...,...,...,...,...
6998,Klinik Kesihatan Telupid,0,0,0,1,,,Telupid,,SABAH
6999,Klinik Kesihatan Terachi,0,0,0,1,2.741806444,102.1356437,Kuala Pilah,,NEGERI SEMBILAN
7000,Klinik Kesihatan Tongod,0,0,0,1,,,Tongod,,SABAH
7002,Klinik Kesihatan Ulu Yam Bharu,0,0,0,1,,,Ulu Selangor,,SELANGOR


In [154]:
merge_df["Clinic"] = merge_df["Clinic"].apply(lambda x: x == 1)
merge_df["Hospital"] = merge_df["Hospital"].apply(lambda x: x == 1)
merge_df["ST"] = merge_df["ST"].apply(lambda x: x == 1)
merge_df["VAC"] = merge_df["VAC"].apply(lambda x: x == 1)

In [155]:
js_data = merge_df.to_dict(orient = 'records')
print(len(js_data))
# print(json.dumps({"data":js_data}, indent=4))
# print(type(js_data))

7085


In [139]:
# output
with open("./JSON/hospital.json", "w") as writer:
    json.dump(js_data, writer, indent=4)